## 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import Word2Vec

C:\Users\JAVI\Miniconda3\envs\dhds\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2. Cargar datos

In [2]:
movies=pd.read_csv('../Data/movies.csv')
ratings=pd.read_csv('../Data/ratings.csv')
links=pd.read_csv('../Data/links.csv')
tags=pd.read_csv('../Data/tags.csv')
imdb=pd.read_csv('../Data/tab_imdb_info.csv',index_col=0)

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [6]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


In [7]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


## 3. Feature Extraction

Corregir id's de imdb:

In [8]:
imdb['imdbId']=imdb['req_imdbid'].apply(lambda x: x[2:]).astype('int64')

In [9]:
pd.merge(links,movies,on='movieId').head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [10]:
imdb.isna().sum()

actors        32
country       12
director      94
req_imdbid     0
year           0
imdbId         0
dtype: int64

In [11]:
imdb.fillna('-',inplace=True)

In [12]:
imdb['actors'].apply(lambda x: len(x.split(','))).unique()

array([4, 2, 3, 1], dtype=int64)

In [13]:
# imdb['director1'] = imdb['director'].apply(lambda x: x.split(',')[0])
# imdb['actor1'] = imdb['actors'].apply(lambda x: x.split(',')[0])
# #imdb['actor2'] = imdb['actors'].apply(lambda x: x.split(',')[1])
# imdb['country1'] = imdb['country'].apply(lambda x: x.split(',')[0])

Creo un dataframe con todos los datos necesarios para crear features por películas:

In [14]:
mov_with_feat = pd.merge(links,imdb,on='imdbId')
mov_with_feat = pd.merge(mov_with_feat,movies,on='movieId')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


In [15]:
mov_with_feat.isna().sum()

movieId       0
imdbId        0
tmdbId        8
actors        0
country       0
director      0
req_imdbid    0
year          0
title         0
genres        0
dtype: int64

In [16]:
mov_with_feat.fillna('-',inplace=True)

In [17]:
mov_with_feat['actors'] = mov_with_feat['actors'].str.replace(' ','')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace(' ','')
mov_with_feat['director'] = mov_with_feat['director'].str.replace(' ','')
mov_with_feat['country'] = mov_with_feat['country'].str.replace(' ','')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy"
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance"
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance"
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


Armado de tokens:

In [18]:
#df1 = mov_with_feat[['actor1','director1','country1']]
df1 = mov_with_feat[['actors','director','country','genres']]
df1 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)
df1 = df1.str.replace(' ','')
df1 = pd.DataFrame({'clean': df1}) 
print(df1.iloc[0].values)
sent = [row.split(',') for row in df1['clean']]
mov_sent = [a for a in zip(imdb['req_imdbid'],sent)]
mov_sent[0:2]

['TomHanks,TimAllen,DonRickles,JimVarney,JohnLasseter,USA,Adventure,Animation,Children,Comedy,Fantasy']


[('tt0114709',
  ['TomHanks',
   'TimAllen',
   'DonRickles',
   'JimVarney',
   'JohnLasseter',
   'USA',
   'Adventure',
   'Animation',
   'Children',
   'Comedy',
   'Fantasy']),
 ('tt0113497',
  ['RobinWilliams',
   'JonathanHyde',
   'KirstenDunst',
   'BradleyPierce',
   'JoeJohnston',
   'USA',
   'Adventure',
   'Children',
   'Fantasy'])]

Entrenar modelo de word2vec de gensim:

In [19]:
model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [20]:
#Representation for director John Lasseter
model.wv['JohnLasseter']

array([-0.00812474,  0.02219038, -0.13629742, -0.06812207, -0.04932716,
       -0.04544505,  0.10206544,  0.07050139, -0.00258269,  0.01271585,
       -0.00541251,  0.05166418,  0.06213424, -0.01605988, -0.05592975,
        0.07479975, -0.05262347,  0.05855098, -0.02091684,  0.02932028,
       -0.08932471, -0.08301935, -0.00633457, -0.09503993, -0.02784147,
        0.0122588 ,  0.05447495,  0.05829696,  0.01714864, -0.00502475,
        0.09962022, -0.00295439, -0.07304174,  0.01579507, -0.02341432,
        0.02143682, -0.02308349, -0.03419221,  0.0127209 ,  0.04806065,
        0.01022443,  0.01177682, -0.0104064 , -0.05515768, -0.03004179,
        0.01579618,  0.0421125 ,  0.00824872, -0.04222525,  0.01727462],
      dtype=float32)

In [21]:
#Get similarity
model.wv.similarity('TomHanks', 'USA')

0.5414548011288299

In [22]:
model.wv['Adventure']

array([-0.62427366,  1.2299356 , -0.6054975 , -1.0669184 , -0.03653602,
       -0.32616565,  0.49666584,  0.9617545 , -0.20517153,  0.09650631,
       -0.04739233,  0.23033707,  0.72469956, -0.53337836,  0.00935083,
        0.730372  , -0.29523665,  0.6030255 , -0.38640106, -0.03389729,
       -0.5156962 , -0.73369634, -0.2238264 , -0.5295261 ,  0.00583082,
        0.07154215,  0.03152158,  0.78821975, -0.21047875, -0.4032773 ,
        0.21127698, -0.4957901 ,  0.2597317 ,  0.04746807,  0.75399   ,
        0.17057098, -0.22494192, -0.5179529 ,  0.23666866, -0.24235468,
        0.24002372,  0.35018936,  0.05686023, -0.5442583 ,  0.151518  ,
        0.31308445,  0.74016404,  0.07349315, -0.25695682,  0.21564281],
      dtype=float32)

In [23]:
model.wv['(nogenreslisted)']

array([-0.00553771,  0.06318007, -0.27084884, -0.14818357, -0.10336877,
       -0.11326849,  0.16832025,  0.12069437, -0.01216182, -0.00884214,
       -0.01904568,  0.06417146,  0.0909134 , -0.02169696, -0.140268  ,
        0.14217748, -0.0669183 ,  0.1190383 , -0.0187173 ,  0.04197485,
       -0.22056851, -0.16658711, -0.02495076, -0.1941906 , -0.08156897,
        0.03293151,  0.11780616,  0.13456377, -0.02307981,  0.01003983,
        0.22208475, -0.03866437, -0.15123016,  0.05031635, -0.03126462,
        0.02344567, -0.02348871, -0.07349367,  0.03798313,  0.08728705,
       -0.00847673,  0.02199434, -0.04332367, -0.11878702, -0.05418347,
       -0.01622499,  0.08162432,  0.02776108, -0.09168603, -0.0288553 ],
      dtype=float32)

In [24]:
imdb.head()

,actors,country,director,req_imdbid,year,imdbId
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,114709
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,113497
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,113228
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,114885
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,113041


In [25]:
imdb.dtypes

actors        object
country       object
director      object
req_imdbid    object
year          object
imdbId         int64
dtype: object

In [26]:
def rep_col(text):
    #print(text)
    rep = []
    for item in text.split(','):
        rep.append(model.wv[item])
    return np.concatenate(rep,axis=0)

In [27]:
def items_rep(item_lst):
    a=0
    for item in item_lst:
        a+=model.wv[item]
    a/=len(item_lst)
    return a



In [28]:
mov_with_feat['actors_rep'] = mov_with_feat['actors'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['director_rep'] = mov_with_feat['director'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['country_rep'] = mov_with_feat['country'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['genres_rep'] = mov_with_feat['genres'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres,actors_rep,director_rep,country_rep,genres_rep
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy","[0.01770277, -0.003399582, -0.2814597, -0.1002...","[-0.008124744, 0.022190383, -0.13629742, -0.06...","[-0.9112063, 1.4076829, -0.58778447, -1.392849...","[-0.6267214, 1.1165192, -0.5401955, -0.9408003..."
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy","[0.021069147, -0.013693997, -0.24132837, -0.08...","[-0.02470821, 0.03644547, -0.12483062, -0.0701...","[-0.9112063, 1.4076829, -0.58778447, -1.392849...","[-0.63809353, 1.0291268, -0.5161161, -0.909790..."
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance","[0.025332935, -0.018919246, -0.28977153, -0.10...","[0.0055389744, -0.0038269097, -0.1363292, -0.0...","[-0.9112063, 1.4076829, -0.58778447, -1.392849...","[-0.5831043, 1.1218148, -0.39607716, -0.857223..."
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance","[0.015292371, -0.014692996, -0.1755847, -0.057...","[0.01967406, 0.0103367325, -0.30541116, -0.121...","[-0.9112063, 1.4076829, -0.58778447, -1.392849...","[-0.55375886, 1.2107646, -0.39301026, -0.93681..."
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy,"[0.027237857, -0.014314956, -0.29298657, -0.10...","[0.0037826144, 0.008747628, -0.087395914, -0.0...","[-0.9112063, 1.4076829, -0.58778447, -1.392849...","[-0.59518737, 1.3939978, -0.51104116, -1.06618..."


In [29]:
# mov_with_feat['actors_rep'][0]

In [30]:
# mov_with_feat['director_rep'][0]

In [31]:
# mov_with_feat['country_rep'][0]

In [32]:
# np.concatenate([mov_with_feat['actors_rep'][0],mov_with_feat['director_rep'][0],mov_with_feat['country_rep'][0]],axis=0)

In [33]:
mov_with_feat['all_feat']=mov_with_feat.apply(lambda x: np.concatenate([x['actors_rep'],x['director_rep'],x['country_rep'],x['genres_rep']],axis=0),axis=1)

In [34]:
mov_with_feat[['movieId','all_feat']].head()

,movieId,all_feat
0,1,"[0.01770277, -0.003399582, -0.2814597, -0.1002..."
1,2,"[0.021069147, -0.013693997, -0.24132837, -0.08..."
2,3,"[0.025332935, -0.018919246, -0.28977153, -0.10..."
3,4,"[0.015292371, -0.014692996, -0.1755847, -0.057..."
4,5,"[0.027237857, -0.014314956, -0.29298657, -0.10..."


In [35]:
a = mov_with_feat['all_feat'][0].reshape(1,-1)
for mov_feat in mov_with_feat['all_feat'][1:]:
    b = mov_feat.reshape(1,-1)
    a = np.concatenate([a,b])

a.shape

(9742, 200)

In [36]:
feature_matrix = a
feature_matrix.shape

(9742, 200)

In [37]:
feat_names = ['f'+str(i+1) for i in range(feature_matrix.shape[1])]

In [38]:
feature_matrix = pd.DataFrame(feature_matrix,index=mov_with_feat['movieId'],columns=feat_names)

In [39]:
feature_matrix.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.017703,-0.003400,-0.281460,-0.100260,-0.103649,-0.087141,0.161378,0.084230,0.011568,0.003587,...,0.247261,0.285972,0.123081,-0.457963,0.133610,0.362561,0.641073,-0.011898,-0.290134,0.283255
2,0.021069,-0.013694,-0.241328,-0.086311,-0.089675,-0.074929,0.142776,0.074475,0.006155,0.004661,...,0.232982,0.304115,0.100542,-0.423954,0.111046,0.374318,0.645290,-0.031280,-0.268557,0.321540
3,0.025333,-0.018919,-0.289772,-0.101136,-0.116739,-0.086364,0.160219,0.086166,0.006061,0.000061,...,0.022048,0.408571,0.186824,-0.509360,0.044562,0.175232,0.552817,0.114041,-0.297396,0.059544
4,0.015292,-0.014693,-0.175585,-0.057780,-0.072190,-0.058601,0.100192,0.047474,0.001760,-0.002279,...,0.022509,0.444965,0.183059,-0.584969,0.061490,0.118276,0.570928,0.172136,-0.308740,-0.008265
5,0.027238,-0.014315,-0.292987,-0.102513,-0.112916,-0.091514,0.169081,0.087958,0.004788,0.000657,...,0.188490,0.339515,0.195722,-0.625431,0.152817,0.224950,0.629804,0.133102,-0.396118,0.057063


In [40]:
feature_matrix.shape

(9742, 200)

## 4. Perfiles de Usuarios

Los mismos se obtienen de multiplicar la matriz user-item (n_users\*n_items) con la matriz de movie_features (n_items\*n_features)

In [41]:
ratings_matrix = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating')
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
ratings_matrix.shape

(610, 9724)

In [43]:
# np.dot(ratings_matrix.values,feature_matrix.values)

In [44]:
len(ratings['movieId'].unique())

9724

In [45]:
len(movies['movieId'].unique())

9742

In [46]:
not_rated = []
for iid in movies['movieId'].unique():
    if iid not in ratings['movieId'].unique():
        not_rated.append(iid)
not_rated

[1076,
 2939,
 3338,
 3456,
 4194,
 5721,
 6668,
 6849,
 7020,
 7792,
 8765,
 25855,
 26085,
 30892,
 32160,
 32371,
 34482,
 85565]

In [47]:
len(not_rated)

18

Hay 18 películas que no tienen puntaje de ningun usuario. Por lo tanto, las sacaremos del dataset para este enfoque, ya que necesitamos los rating de los usuarios para las peliculas.

In [48]:
feature_matrix = feature_matrix.drop(index=not_rated)
feature_matrix.shape

(9724, 200)

In [49]:
user_profiles = np.dot(ratings_matrix.values,feature_matrix.values)

In [50]:
user_profiles = pd.DataFrame(user_profiles,index=ratings_matrix.index,columns=feature_matrix.columns)

In [51]:
user_profiles.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200
userId,,,,,,,,,,,,,,,,,,,,,
1,16.376331,-6.016376,-218.366150,-79.910255,-85.704251,-67.657745,125.617381,65.641501,4.167418,0.251680,...,102.004678,372.528201,107.283146,-540.037994,85.950019,182.486866,569.064701,129.783166,-246.293454,50.218178
2,2.332451,-0.702275,-30.193499,-11.129003,-11.946041,-9.552445,17.439489,9.173270,0.543981,0.109895,...,5.552850,45.003221,12.916609,-65.171291,8.049507,10.654608,60.163407,21.930993,-27.923467,-6.139881
3,0.019762,1.615843,-16.722290,-7.477221,-6.406284,-5.615944,9.854203,6.312979,0.003089,0.169851,...,10.474085,34.076752,5.992240,-50.613416,7.701672,15.933887,51.712762,13.550454,-18.902888,3.380964
4,11.621038,-4.392411,-149.011095,-54.249702,-58.928057,-46.508871,86.118611,44.244706,3.259344,-0.127604,...,46.399574,310.065970,105.204626,-440.910602,59.806966,92.491736,423.115547,134.270256,-209.472136,-15.519950
5,2.622840,-0.755940,-34.679470,-12.858020,-13.662002,-10.894320,20.005054,10.311378,0.637858,0.043308,...,9.507320,64.617630,19.774654,-88.212147,11.183926,21.333410,88.000653,25.355950,-40.274676,0.434457


In [52]:
user_profiles.shape

(610, 200)

### Lista de películas vistas por usuario

In [53]:
ratings_sorted = ratings.sort_values(by='timestamp')
movies_by_user = pd.DataFrame(ratings_sorted.groupby('userId')['movieId'].apply(list))
movies_by_user.head()

,movieId
userId,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744..."
2,"[318, 79132, 131724, 115713, 99114, 112552, 35..."
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3..."
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5..."
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ..."


## 5. Recomendaciones por usuario

In [56]:
def get_user_recommendations(uid,num):
    C = cosine_similarity(np.atleast_2d(user_profiles.loc[uid]), feature_matrix.values)
    R = np.argsort(C)[:, ::-1]
    recommendations = [i for i in R[0] if i not in movies_by_user.loc[uid].values]
    return movies['title'][recommendations].head(num)

In [57]:
get_user_recommendations(uid=1,num=10)

777     20,000 Leagues Under the Sea (1954)
7724                        Puncture (2011)
1827                    Faculty, The (1998)
4487               Man's Best Friend (1993)
6295                 Bow, The (Hwal) (2005)
1487     Back to the Future Part III (1990)
3880          Look Who's Talking Now (1993)
2257           House on Haunted Hill (1999)
325                        Mask, The (1994)
6077             Memoirs of a Geisha (2005)
Name: title, dtype: object

In [60]:
get_user_recommendations(uid=154,num=10)

4416                                     Babe, The (1992)
7609                             Midnight in Paris (2011)
8677                       The Diary of Anne Frank (2009)
8457    Internet's Own Boy: The Story of Aaron Swartz,...
9695            Fred Armisen: Standup for Drummers (2018)
1556                                  Return to Oz (1985)
5880                                 King's Ransom (2005)
8671                  Bill Burr: Why Do I Do This? (2008)
4650                                Big Empty, The (2003)
224             Star Wars: Episode IV - A New Hope (1977)
Name: title, dtype: object

In [61]:
get_user_recommendations(uid=610,num=10)

8040    Fantastic Fear of Everything, A (2012)
3921               24 Hour Party People (2002)
8219                              Turbo (2013)
6441             I Think I Love My Wife (2007)
1722                            Beloved (1998)
2542                  Raise the Titanic (1980)
1487        Back to the Future Part III (1990)
6664                        Cloverfield (2008)
6295                    Bow, The (Hwal) (2005)
5555                    Nuns on the Run (1990)
Name: title, dtype: object